Query rari and vesta subgraphs for OHM cross-chain liquidations. Compile into daily dataset for further analysis

In [1]:
import matplotlib.pyplot as plt
import pandas as pd

from subgrounds.subgrounds import Subgrounds
from subgrounds.subgraph import SyntheticField

In [2]:
def get_liq_data(endpoint: str) -> pd.DataFrame:
    """
    Get liquidation data from a standardized messari borrowing/lending subgraph.
    standardization schema - https://github.com/messari/subgraphs/blob/master/schema-lending.graphql
    """
    sub = Subgrounds()

    # load subground endpoint
    liq_sub = sub.load_subgraph(endpoint)

    # subgrounds query path
    liqs = liq_sub.Query.liquidates

    # insert datetime synthetic field
    liq_sub.Liquidate.datetime = SyntheticField.datetime_of_timestamp(liq_sub.Liquidate.timestamp)

    liq_data = liqs(
        orderBy=liqs.timestamp,
        orderDirection='desc',
        first=10000000 # set to very large number to get all data
    )

    # add subgraph fields to dataframe columns
    liq_df = sub.query_df([
        liq_data.datetime,
        liq_data.timestamp,
        liq_data.hash,
        liq_data.blockNumber,
        liq_data.liquidator.id,
        liq_data.liquidatee.id,
        liq_data.market.id,
        liq_data.market.name,
        liq_data.amount,
        liq_data.amountUSD,
        liq_data.profitUSD
    ])

    return liq_df

    

In [3]:
# get liquidation data from subgraphs
rari_liq_df = get_liq_data('https://api.thegraph.com/subgraphs/name/messari/rari-fuse-ethereum') # https://thegraph.com/hosted-service/subgraph/messari/rari-fuse-ethereum
vesta_liq_df = get_liq_data('https://api.thegraph.com/subgraphs/name/corerouter/vesta-finance') # https://thegraph.com/hosted-service/subgraph/corerouter/vesta-finance


In [4]:
# preprocess liquidation dfs
def preprocess_liq_df(liq_df: pd.DataFrame, filter: list[str], source: str) -> pd.DataFrame:
    """
    Preprocess liquidation dataframes:
    - filter by market
    - convert datetime column to daily frequency
    - aggregate daily liquidation amounts
    """
    # filter for liquidation tokens of interest
    liq_df_filter = liq_df[liq_df['liquidates_market_name'].str.contains(filter)]

    # convert liquidates_datetime from str to datetime type
    liq_df_filter['liquidates_datetime'] = pd.to_datetime(liq_df_filter['liquidates_datetime'])

    # set liquidates_datetime as index
    liq_df_filter = liq_df_filter.set_index('liquidates_datetime')

    # aggregate datetime to daily
    liq_df_filter.index = liq_df_filter.index.floor('D')
    daily_liq_df_filter = liq_df_filter.groupby('liquidates_datetime').agg({'liquidates_amountUSD': sum})

    # add source of the liquidation data
    daily_liq_df_filter['source'] = source

    return daily_liq_df_filter

In [5]:
# #filter rari pools for ohm token
ohm_keywords =['Olympus', 'OHM', 'ohm', 'olympus', 'Governance OHM']
pattern = '|'.join(ohm_keywords)

In [6]:
rari_daily_df = preprocess_liq_df(rari_liq_df, pattern, 'rari')
vesta_daily_df = preprocess_liq_df(vesta_liq_df, pattern, 'vesta')

/tmp/ipykernel_18384/3629167007.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  liq_df_filter['liquidates_datetime'] = pd.to_datetime(liq_df_filter['liquidates_datetime'])
/tmp/ipykernel_18384/3629167007.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  liq_df_filter['liquidates_datetime'] = pd.to_datetime(liq_df_filter['liquidates_datetime'])


In [7]:
# combine liquidation dataframes
combined_df = pd.concat([vesta_daily_df, rari_daily_df])

In [8]:
combined_df = combined_df.groupby('liquidates_datetime').agg({'liquidates_amountUSD': sum})

In [9]:
combined_df.head(5)

,liquidates_amountUSD
liquidates_datetime,
2021-07-02,2228.809851
2021-07-05,21862.096450
2021-07-06,202841.753227
2021-07-08,1317.510494
2021-07-13,4946.100307


In [10]:
# save to csv
combined_df.to_csv("../../tda_ohm_analysis/data/ohm_liquidations.csv")